In [1]:
import os
from glob import glob
# from transformers import Wav2Vec2FeatureExtractor, Wav2Vec2Processor, Wav2Vec2ForCTC
from transformers import AutoTokenizer, AutoFeatureExtractor, AutoModelForCTC, Wav2Vec2Processor
import torchaudio
import soundfile as sf
import librosa
import torch.functional as F
import torch.nn as nn
from IPython.display import Audio, display, Markdown
import torch

import pandas as pd
import random

In [2]:
# import model, feature extractor, tokenizer
model = AutoModelForCTC.from_pretrained("facebook/wav2vec2-base-960h")
tokenizer = Wav2Vec2Processor.from_pretrained("facebook/wav2vec2-base-960h")
feature_extractor = AutoFeatureExtractor.from_pretrained("facebook/wav2vec2-base-960h")

Some weights of Wav2Vec2ForCTC were not initialized from the model checkpoint at facebook/wav2vec2-base-960h and are newly initialized: ['wav2vec2.masked_spec_embed']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Ignored unknown kwarg option normalize
Ignored unknown kwarg option normalize
Ignored unknown kwarg option normalize
Ignored unknown kwarg option normalize


In [3]:
metadata = {
    "angry": sorted(glob(r"data/audio-emotions/Angry/*.wav")),
    "sad": sorted(glob(r"data/audio-emotions/Sad/*.wav")),
    "disgusted": sorted(glob(r"data/audio-emotions/Disgusted/*.wav")),
    "fearful": sorted(glob(r"data/audio-emotions/Fearful/*.wav")),
    "happy": sorted(glob(r"data/audio-emotions/Happy/*.wav")),
    "neutral": sorted(glob(r"data/audio-emotions/Neutral/*.wav")),
    "surprised": sorted(glob(r"data/audio-emotions/Surprised/*.wav")),
}

TRAIN_SPLIT = 0.8

train_meta = {
    "angry": metadata["angry"][:int(TRAIN_SPLIT * len(metadata["angry"]))],
    "sad": metadata["sad"][:int(TRAIN_SPLIT * len(metadata["sad"]))],
    "disgusted": metadata["disgusted"][:int(TRAIN_SPLIT * len(metadata["disgusted"]))],
    "fearful": metadata["fearful"][:int(TRAIN_SPLIT * len(metadata["fearful"]))],
    "happy": metadata["happy"][:int(TRAIN_SPLIT * len(metadata["happy"]))],
    "neutral": metadata["neutral"][:int(TRAIN_SPLIT * len(metadata["neutral"]))],
    "surprised": metadata["surprised"][:int(TRAIN_SPLIT * len(metadata["surprised"]))],
}

test_meta = {
    "angry": metadata["angry"][int(TRAIN_SPLIT * len(metadata["angry"])):],
    "sad": metadata["sad"][int(TRAIN_SPLIT * len(metadata["sad"])):],
    "disgusted": metadata["disgusted"][int(TRAIN_SPLIT * len(metadata["disgusted"])):],
    "fearful": metadata["fearful"][int(TRAIN_SPLIT * len(metadata["fearful"])):],
    "happy": metadata["happy"][int(TRAIN_SPLIT * len(metadata["happy"])):],
    "neutral": metadata["neutral"][int(TRAIN_SPLIT * len(metadata["neutral"])):],
    "surprised": metadata["surprised"][int(TRAIN_SPLIT * len(metadata["surprised"])):],
}
        

In [4]:
train_df = []
test_df = []

for emotion in train_meta:
    for file in train_meta[emotion]:
        train_df.append((file, emotion))
        
for emotion in test_meta:
    for file in test_meta[emotion]:
        test_df.append((file, emotion))
        
random.shuffle(train_df)
random.shuffle(test_df)


In [5]:
# create pandas dataframes
train_df = pd.DataFrame(train_df, columns=["file", "emotion"])
test_df = pd.DataFrame(test_df, columns=["file", "emotion"])

# apply feature extractor to all files
# train_df = train_df.head()
# test_df = test_df.head()
# flattener = nn.Flatten()
def extract_features(file):
    # waveform, sample_rate = sf.read(file)
    waveform, sample_rate = librosa.load(file, sr=16000)
    # input_values = tokenizer(waveform, sampling_rate=sample_rate, return_tensors="pt").input_values
    # sampling_rate=sample_rate
    features = feature_extractor(waveform, sampling_rate=sample_rate, return_tensors="np").input_values
    
    # features = features[0]
    print(f"{features.shape = }")
    return features.tolist()


In [6]:

train_df = train_df.head()
test_df = test_df.head()
train_df["features"] = train_df["file"].apply(extract_features)
test_df["features"] = test_df["file"].apply(extract_features)
train_df.head()

features.shape = (1, 81148)
features.shape = (1, 56056)
features.shape = (1, 40040)
features.shape = (1, 37904)
features.shape = (1, 36303)
features.shape = (1, 37828)
features.shape = (1, 31738)
features.shape = (1, 33066)
features.shape = (1, 27471)
features.shape = (1, 35203)


,file,emotion,features
0,data/audio-emotions/Sad/03-02-04-01-01-01-08.wav,sad,"[[0.0013608308508992195, 0.0009191426215693355..."
1,data/audio-emotions/Disgusted/1049_IWL_DIS_XX.wav,disgusted,"[[-0.05532725155353546, -0.05045466870069504, ..."
2,data/audio-emotions/Sad/1034_ITS_SAD_XX.wav,sad,"[[0.24642083048820496, 0.34120050072669983, 0...."
3,data/audio-emotions/Disgusted/1049_IEO_DIS_MD.wav,disgusted,"[[0.10805115848779678, 0.11462341994047165, 0...."
4,data/audio-emotions/Neutral/1043_MTI_NEU_XX.wav,neutral,"[[0.03921518474817276, 0.02377452328801155, -0..."


In [7]:
# transcription
def transcribe(features):
    input_values = torch.tensor(features)
    logits = model(input_values).logits[0]
    predicted_ids = torch.argmax(logits, dim=-1)
    outputs = tokenizer.decode(predicted_ids, output_word_offsets=True)
    time_offset = model.config.inputs_to_logits_ratio / feature_extractor.sampling_rate
    
    word_offsets = [
        { "word": d["word"],
            "start_time": round(d["start_offset"] * time_offset, 2),
            "end_time": round(d["end_offset"] * time_offset, 2),
        }
        for d in outputs.word_offsets
    ]

    return word_offsets

In [8]:
predictions_df = train_df.copy()
predictions_df["transcription"] = predictions_df["features"].apply(transcribe)
predictions_df.head()

,file,emotion,features,transcription
0,data/audio-emotions/Sad/03-02-04-01-01-01-08.wav,sad,"[[0.0013608308508992195, 0.0009191426215693355...","[{'word': 'KISER', 'start_time': 1.04, 'end_ti..."
1,data/audio-emotions/Disgusted/1049_IWL_DIS_XX.wav,disgusted,"[[-0.05532725155353546, -0.05045466870069504, ...","[{'word': 'I', 'start_time': 0.76, 'end_time':..."
2,data/audio-emotions/Sad/1034_ITS_SAD_XX.wav,sad,"[[0.24642083048820496, 0.34120050072669983, 0....","[{'word': 'I', 'start_time': 0.34, 'end_time':..."
3,data/audio-emotions/Disgusted/1049_IEO_DIS_MD.wav,disgusted,"[[0.10805115848779678, 0.11462341994047165, 0....","[{'word': 'TELEVEN', 'start_time': 0.68, 'end_..."
4,data/audio-emotions/Neutral/1043_MTI_NEU_XX.wav,neutral,"[[0.03921518474817276, 0.02377452328801155, -0...","[{'word': 'MAY', 'start_time': 0.22, 'end_time..."


In [9]:
print(f"TRANSCRIPTION: {' '.join([ item['word'] for item in predictions_df['transcription'].iloc[2]])}")
Audio(train_df["file"].iloc[2], autoplay=True)

TRANSCRIPTION: I THINK OUR SINNISTER BEFORE


In [10]:
train_df["emotion"].iloc[2]

'sad'

In [11]:
train_df.iloc[0]["features"]

[[0.0013608308508992195,
  0.0009191426215693355,
  0.0004234753723721951,
  0.0007371707470156252,
  0.00042317245970480144,
  -2.0562636564136483e-05,
  0.00014149710477795452,
  6.553858111146837e-05,
  9.894639515550807e-05,
  9.052270615939051e-05,
  8.295653242385015e-05,
  0.00010124308028025553,
  7.559988443972543e-05,
  0.00010632809426169842,
  7.210242620203644e-05,
  0.00010869001562241465,
  7.056007598293945e-05,
  0.00010965023102471605,
  7.001090853009373e-05,
  0.00010991353337885812,
  6.993556598899886e-05,
  0.00010987168207066134,
  7.005026418482885e-05,
  0.00010970434959745035,
  7.026696403045207e-05,
  0.00010942368680844083,
  7.06473074387759e-05,
  0.00010889057739404961,
  7.1405251219403e-05,
  0.00010781105083879083,
  7.29215462342836e-05,
  0.0001057109038811177,
  7.578970689792186e-05,
  0.00010184168058913201,
  8.096470264717937e-05,
  9.495135600445792e-05,
  9.01401654118672e-05,
  8.265000360552222e-05,
  0.00010689707414712757,
  5.9177051298

In [12]:
# save to csv
# train_df.to_csv("data/audio_emotions-train.csv", index=False)
# test_df.to_csv("data/audio_emotions-test.csv", index=False)

---

## Trying to use loading script to load entire dataset

In [1]:
from local_dataset import AudioEmotionsDataset
from IPython.display import Audio, display, Markdown

In [2]:
local_dataset = AudioEmotionsDataset("data/audio-emotions")

Some weights of Wav2Vec2ForCTC were not initialized from the model checkpoint at facebook/wav2vec2-base-960h and are newly initialized: ['wav2vec2.masked_spec_embed']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Ignored unknown kwarg option normalize
Ignored unknown kwarg option normalize
Ignored unknown kwarg option normalize
Ignored unknown kwarg option normalize


In [3]:
waveform, X, y = local_dataset.get_train(0)
waveform.shape, X.shape, y

((43777,), torch.Size([1, 43777]), 0)

In [4]:
waveform[0], X[0][0]

(-0.003112793, tensor(-0.0755))

In [5]:
_, transcription = local_dataset.transcribe(X)

print(f"TRANSCRIPTION: {transcription}")
Audio(waveform,  rate=16000, autoplay=True)

TRANSCRIPTION: IT'S ELEVEN O'CLOCK


In [7]:
label = local_dataset.num_to_label(y)
label

'angry'